In [1]:
# load tf stuff
import urllib

import numpy as np
import tensorflow as tf
tf.enable_v2_behavior()

from thesis_v2.blocks.rcnn_sat import bl_net


/opt/conda/envs/leelab/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/leelab/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/leelab/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/leelab/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [2]:
input_layer = tf.keras.layers.Input((128, 128, 3))
model = bl_net(input_layer, classes=565, cumulative_readout=True)

In [3]:
_, msg = urllib.request.urlretrieve(
    'https://osf.io/9td5p/download', 'bl_ecoset.h5')
print(msg)

Server: nginx
Date: Sun, 02 Feb 2020 01:09:47 GMT
Content-Type: application/octet-stream
Content-Length: 112560228
X-Waterbutler-Request-Id: 83787be2-26c6-4cd5-bf7f-27c0c7b8554e
Content-Disposition: attachment; filename="bl_ecoset.h5"; filename*=UTF-8''bl_ecoset.h5
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Pragma: no-cache
Via: 1.1 google
Alt-Svc: clear
Connection: close




In [4]:
model.load_weights('bl_ecoset.h5')

In [5]:
get_layer_activation = tf.keras.backend.function(
    [model.input],
    [
        model.get_layer('ReLU_Layer_6_Time_0').output,
        model.get_layer('ReLU_Layer_6_Time_1').output,
        model.get_layer('ReLU_Layer_6_Time_2').output,
        model.get_layer('ReLU_Layer_6_Time_3').output,
        model.get_layer('ReLU_Layer_6_Time_4').output,
        model.get_layer('ReLU_Layer_6_Time_5').output,
        model.get_layer('ReLU_Layer_6_Time_6').output,
        model.get_layer('ReLU_Layer_6_Time_7').output,
    ]
)

In [6]:
np.random.seed(0)
random_img = np.random.random_sample((1, 128, 128, 3))*2 - 1
random_img = random_img.astype(np.float32)

In [7]:
tf_out = get_layer_activation(random_img)

In [8]:
from pickle import dump

with open('debug_rcnn_basic_kriegeskorte_tf.pkl', 'wb') as f_out:
    dump(tf_out, f_out)